<!-- # Car Color Detection Model Training
This notebook trains a model to detect car colors in traffic and count cars and people.
Red rectangles for blue cars, blue rectangles for other color cars. -->

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO

In [2]:
# Car colors
car_colors = ['beige', 'black', 'blue', 'brown','gold', 'gray', 'green', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']

# Create dataset structure
dataset_path = r'C:\Users\sarva\Emotion_detection-main\datasets\car_color_data'

for color in car_colors:
    os.makedirs(f'{dataset_path}/train/{color}', exist_ok=True)
    os.makedirs(f'{dataset_path}/test/{color}', exist_ok=True)

# Traffic images with cars and people
os.makedirs(f'{dataset_path}/traffic_images', exist_ok=True)

In [3]:
# Image parameters
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 20

# Data augmentation for color classification
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [4]:
# Load pre-trained YOLO model for car and person detection
yolo_model = YOLO('yolov8n.pt') 

# Classes we're interested in (COCO dataset)
CAR_CLASSES = [2, 5, 7]  # car, bus, truck
PERSON_CLASS = 0  # person

print('YOLO model loaded for object detection.')

YOLO model loaded for object detection.


In [5]:
def analyze_traffic_scene(image, color_model, yolo_model):
    """
    Analyze traffic scene:
    - Detect cars and people
    - Classify car colors
    - Draw red rectangles for blue cars, blue rectangles for other cars
    - Count cars and people
    """
    results = yolo_model(image)
    
    car_count = 0
    people_count = 0
    blue_cars = 0
    other_cars = 0
    
    # Create a copy of the image for drawing
    output_image = image.copy()
    
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for box in boxes:
                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                
                if confidence > 0.5:  # Confidence threshold
                    if class_id in CAR_CLASSES:
                        # Extract car region for color classification
                        car_region = image[y1:y2, x1:x2]
                        if car_region.size > 0:
                            # Preprocess for color model
                            car_resized = cv2.resize(car_region, (IMG_HEIGHT, IMG_WIDTH))
                            car_normalized = np.expand_dims(car_resized / 255.0, axis=0)
                            
                            # Predict color
                            color_pred = color_model.predict(car_normalized, verbose=0)
                            predicted_color_idx = np.argmax(color_pred[0])
                            predicted_color = car_colors[predicted_color_idx]
                            confidence_color = float(np.max(color_pred[0]))
                            
                            # Draw rectangle based on color
                            if predicted_color == 'blue':
                                color_rect = (0, 0, 255)  # Red rectangle for blue cars
                                blue_cars += 1
                            else:
                                color_rect = (255, 0, 0)  # Blue rectangle for other cars
                                other_cars += 1
                            
                            # Draw bounding box
                            cv2.rectangle(output_image, (x1, y1), (x2, y2), color_rect, 2)
                            
                            # Add label
                            label = f'{predicted_color}: {confidence_color:.2f}'
                            cv2.putText(output_image, label, (x1, y1-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_rect, 2)
                            
                            car_count += 1
                    
                    elif class_id == PERSON_CLASS:
                        # Draw green rectangle for people
                        cv2.rectangle(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(output_image, 'Person', (x1, y1-10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        people_count += 1
    
    # Add statistics text
    stats_text = [
        f'Total Cars: {car_count}',
        f'Blue Cars: {blue_cars}',
        f'Other Cars: {other_cars}',
        f'People: {people_count}'
    ]
    
    for i, text in enumerate(stats_text):
        cv2.putText(output_image, text, (10, 30 + i*30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return {
        'image': output_image,
        'car_count': car_count,
        'blue_cars': blue_cars,
        'other_cars': other_cars,
        'people_count': people_count
    }

print('Traffic analysis function created.')

Traffic analysis function created.


In [6]:
train_generator = train_datagen.flow_from_directory(
    f'{dataset_path}/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    f'{dataset_path}/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 5476 images belonging to 15 classes.
Found 1363 images belonging to 15 classes.


In [1]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import os
import shutil
from collections import Counter

# Enable optimizations
tf.config.optimizer.set_jit(True)
mixed_precision.set_global_policy('mixed_float16')

# Configuration
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
EPOCHS = 20
LEARNING_RATE = 1e-4

# Dataset paths
dataset_path = r'C:\Users\sarva\Emotion_detection-main\datasets\car_color_data'
train_path = f'{dataset_path}/train'
val_path = f'{dataset_path}/val'

def synchronize_train_val_folders(train_dir, val_dir):
    """Ensure train and val directories have exactly the same classes"""
    
    print(" Synchronizing train/validation folders...")
    
    if not os.path.exists(train_dir):
        print(f" Training directory not found: {train_dir}")
        return []
    
    if not os.path.exists(val_dir):
        print(f" Validation directory not found: {val_dir}")
        return []
    
    # Get class directories
    train_classes = {d for d in os.listdir(train_dir) 
                    if os.path.isdir(os.path.join(train_dir, d)) and 
                    len([f for f in os.listdir(os.path.join(train_dir, d)) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]) > 0}
    
    val_classes = {d for d in os.listdir(val_dir) 
                  if os.path.isdir(os.path.join(val_dir, d))}
    
    print(f"Training classes ({len(train_classes)}): {sorted(train_classes)}")
    print(f"Validation classes ({len(val_classes)}): {sorted(val_classes)}")
    
    # Find mismatches
    only_in_train = train_classes - val_classes
    only_in_val = val_classes - train_classes
    
    if only_in_train:
        print(f" Classes only in training: {only_in_train}")
        # Create empty validation folders
        for cls in only_in_train:
            val_cls_path = os.path.join(val_dir, cls)
            os.makedirs(val_cls_path, exist_ok=True)
            print(f"   Created validation folder: {cls}")
    
    if only_in_val:
        print(f" Classes only in validation: {only_in_val}")
        # Remove from validation or create in training
        for cls in only_in_val:
            val_cls_path = os.path.join(val_dir, cls)
            # Count images in this validation class
            images = [f for f in os.listdir(val_cls_path) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if len(images) > 0:
                print(f"   Moving {len(images)} images from val/{cls} to train/{cls}")
                # Create training folder and move images
                train_cls_path = os.path.join(train_dir, cls)
                os.makedirs(train_cls_path, exist_ok=True)
                
                for img in images:
                    src = os.path.join(val_cls_path, img)
                    dst = os.path.join(train_cls_path, img)
                    shutil.move(src, dst)
            else:
                print(f"   Removing empty validation folder: {cls}")
                shutil.rmtree(val_cls_path)
    
    # Get final synchronized classes
    final_classes = sorted(train_classes | only_in_val)
    print(f" Synchronized to {len(final_classes)} classes: {final_classes}")
    
    return final_classes

def create_validation_split_from_training(train_dir, val_dir, split_ratio=0.2):
    """Create validation set by moving images from training"""
    
    print(f" Creating validation split ({split_ratio*100}% from training)...")
    
    moved_count = 0
    for class_name in os.listdir(train_dir):
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        
        if os.path.isdir(train_class_dir):
            # Get all images
            images = [f for f in os.listdir(train_class_dir) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if len(images) > 1:  # Only split if more than 1 image
                # Calculate validation count
                val_count = max(1, int(len(images) * split_ratio))
                val_images = np.random.choice(images, val_count, replace=False)
                
                # Create validation directory
                os.makedirs(val_class_dir, exist_ok=True)
                
                # Move images
                for img in val_images:
                    src = os.path.join(train_class_dir, img)
                    dst = os.path.join(val_class_dir, img)
                    shutil.move(src, dst)
                    moved_count += 1
                
                print(f"   {class_name}: moved {len(val_images)} images to validation")
    
    print(f" Moved {moved_count} images to validation")

def create_optimized_dataset(data_dir, batch_size=BATCH_SIZE, training=True):
    """Create optimized dataset with automatic class detection"""
    
    AUTOTUNE = tf.data.AUTOTUNE
    
    # Load dataset with explicit class detection
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=IMG_SIZE,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=training
    )
    
    # Get class names from dataset
    class_names = dataset.class_names
    print(f"Dataset loaded from {data_dir}: {len(class_names)} classes")
    
    # Normalize images
    def normalize_img(image, label):
        image = tf.cast(image, tf.float32) / 255.0
        return image, label
    
    dataset = dataset.map(normalize_img, num_parallel_calls=AUTOTUNE)
    
    if training:
        # Data augmentation
        def augment(image, label):
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, 0.1)
            image = tf.image.random_contrast(image, 0.9, 1.1)
            return image, label
        
        dataset = dataset.map(augment, num_parallel_calls=AUTOTUNE)
    
    # Performance optimizations
    dataset = dataset.cache()
    if training:
        dataset = dataset.shuffle(1000)
    dataset = dataset.prefetch(AUTOTUNE)
    
    return dataset, class_names

def create_model(num_classes):
    """Create optimized model"""
    
    # Input
    input_layer = tf.keras.Input(shape=(*IMG_SIZE, 3))
    
    # Base model
    base_model = MobileNetV2(
        input_tensor=input_layer,
        weights='imagenet',
        include_top=False,
        pooling='avg'
    )
    
    base_model.trainable = False
    
    # Classification head
    x = base_model.output
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    # Output layer with correct number of classes
    predictions = Dense(num_classes, activation='softmax', dtype=tf.float32)(x)
    
    model = Model(inputs=input_layer, outputs=predictions)
    return model

def main():
    
    #  Synchronize folders
    synchronized_classes = synchronize_train_val_folders(train_path, val_path)
    
    if not synchronized_classes:
        print(" No classes found or synchronization failed")
        return
    
    # Check if validation folder is too empty
    val_image_count = 0
    if os.path.exists(val_path):
        for cls in os.listdir(val_path):
            cls_path = os.path.join(val_path, cls)
            if os.path.isdir(cls_path):
                images = [f for f in os.listdir(cls_path) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                val_image_count += len(images)
    

    if val_image_count < 100: 
        print(f" Only {val_image_count} validation images found")
        create_validation_split_from_training(train_path, val_path)
    
    # Create datasets
    print("\n=== Creating Datasets ===")
    
    try:
        train_dataset, train_class_names = create_optimized_dataset(train_path, training=True)
        val_dataset, val_class_names = create_optimized_dataset(val_path, training=False)
        
        print(f"Training classes: {train_class_names}")
        print(f"Validation classes: {val_class_names}")
        
        # Verify class alignment
        if train_class_names != val_class_names:
            print(" ERROR: Class mismatch between train and validation!")
            print(f"Train: {set(train_class_names) - set(val_class_names)}")
            print(f"Val: {set(val_class_names) - set(train_class_names)}")
            return
        
        num_classes = len(train_class_names)
        print(f" Datasets synchronized with {num_classes} classes")
        
    except Exception as e:
        print(f" Dataset creation failed: {e}")
        return
    
    # Create and compile model
    print("\n=== Creating Model ===")
    
    model = create_model(num_classes)
    
    # Mixed precision optimizer
    optimizer = Adam(learning_rate=LEARNING_RATE)
    if mixed_precision.global_policy().name == 'mixed_float16':
        optimizer = mixed_precision.LossScaleOptimizer(optimizer)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Model architecture:")
    model.summary()
    
    # Setup callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath='car_color_model_synchronized.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train model
    print("\n=== Starting Training ===")
    
    try:
        history = model.fit(
            train_dataset,
            validation_data=val_dataset,
            epochs=EPOCHS,
            callbacks=callbacks,
            verbose=1
        )
        
        print(" Training completed successfully!")
        
        # Save class names
        import json
        with open('car_color_classes.json', 'w') as f:
            json.dump(train_class_names, f)
        
        print("Model and class names saved successfully!")
        return model, history
        
    except Exception as e:
        print(f" Training failed: {e}")
        return None

if __name__ == "__main__":
    tf.random.set_seed(42)
    np.random.seed(42)
    
    result = main()
    
    if result:
        print("\n Training completed successfully!")
    else:
        print("\n Training failed. Check error messages above.")


 Synchronizing train/validation folders...
Training classes (15): ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']
Validation classes (15): ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']
 Synchronized to 15 classes: ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']

=== Creating Datasets ===
Found 6931 files belonging to 15 classes.
Dataset loaded from C:\Users\sarva\Emotion_detection-main\datasets\car_color_data/train: 15 classes
Found 1459 files belonging to 15 classes.
Dataset loaded from C:\Users\sarva\Emotion_detection-main\datasets\car_color_data/val: 15 classes
Training classes: ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']
Validation classes: ['bei

C:\Users\sarva\AppData\Local\Temp\ipykernel_13744\3803945393.py:178: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Model architecture:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,051,087 (11.64 MB)

 Trainable params: 792,079 (3.02 MB)

 Non-trainable params: 2,259,008 (8.62 MB)


=== Starting Training ===
Epoch 1/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.0935 - loss: 3.1899
Epoch 1: val_accuracy improved from None to 0.37286, saving model to car_color_model_synchronized.keras
109/109 ━━━━━━━━━━━━━━━━━━━━ 1083s 10s/step - accuracy: 0.1327 - loss: 2.9381 - val_accuracy: 0.3729 - val_loss: 2.2120 - learning_rate: 1.0000e-04
Epoch 2/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.2750 - loss: 2.3607
Epoch 2: val_accuracy improved from 0.37286 to 0.49280, saving model to car_color_model_synchronized.keras
109/109 ━━━━━━━━━━━━━━━━━━━━ 1056s 10s/step - accuracy: 0.3005 - loss: 2.2494 - val_accuracy: 0.4928 - val_loss: 1.7922 - learning_rate: 1.0000e-04
Epoch 3/20
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.3678 - loss: 1.9767
Epoch 3: val_accuracy improved from 0.49280 to 0.56134, saving model to car_color_model_synchronized.keras
109/109 ━━━━━━━━━━━━━━━━━━━━ 1075s 10s/step - accuracy: 0.3819 - loss: 1.9269 - val_accuracy: 0.5613 - v

In [ ]:
color_model.save('car_color_detection_model.h5')
print('Color classification model saved successfully!')